<a href="https://www.kaggle.com/code/palpriyanshu/predicting-student-test-marks?scriptVersionId=292950710" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e1/sample_submission.csv
/kaggle/input/playground-series-s6e1/train.csv
/kaggle/input/playground-series-s6e1/test.csv


In [2]:
import pandas as pd
import numpy as np

train=pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")
test=pd.read_csv("/kaggle/input/playground-series-s6e1/test.csv")
sub=pd.read_csv("/kaggle/input/playground-series-s6e1/sample_submission.csv")
print(train.shape,test.shape)
print(train.head())

(630000, 13) (270000, 12)
   id  age  gender   course  study_hours  class_attendance internet_access  \
0   0   21  female     b.sc         7.91              98.8              no   
1   1   18   other  diploma         4.95              94.8             yes   
2   2   20  female     b.sc         4.68              92.6             yes   
3   3   19    male     b.sc         2.00              49.5             yes   
4   4   23    male      bca         7.65              86.9             yes   

   sleep_hours sleep_quality   study_method facility_rating exam_difficulty  \
0          4.9       average  online videos             low            easy   
1          4.7          poor     self-study          medium        moderate   
2          5.8          poor       coaching            high        moderate   
3          8.3       average    group study            high        moderate   
4          9.6          good     self-study            high            easy   

   exam_score  
0        78.3 

In [3]:
def simple_features(df):

    df = df.copy()

    df["study_x_attendance"] = df["study_hours"] * df["class_attendance"]
    df["sleep_x_study"]      = df["sleep_hours"] * df["study_hours"]
    df["age_x_study"]        = df["age"] * df["study_hours"]

    return df


train = simple_features(train)
test  = simple_features(test)

print("features added")
print(train.columns)


features added
Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score',
       'study_x_attendance', 'sleep_x_study', 'age_x_study'],
      dtype='object')


In [4]:
cat_cols=[
    'gender','course','internet_access','sleep_quality',
    'study_method','exam_difficulty','facility_rating'
]

for c in cat_cols:
    train[c]=train[c].astype("category")
    test[c]=test[c].astype("category")

print("categorical converted")

categorical converted


In [5]:
X=train.drop(["id","exam_score"], axis=1)
y=train["exam_score"]

test_x=test.drop(["id"], axis=1)
print(X.shape, test_x.shape)

(630000, 14) (270000, 14)


In [6]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)

oof = np.zeros(len(train))
preds = np.zeros(len(test))

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'seed': 42
}

for tr, va in kf.split(X, y):

    tr_x, tr_y = X.iloc[tr], y.iloc[tr]
    va_x, va_y = X.iloc[va], y.iloc[va]

    model = lgb.LGBMRegressor(**params, n_estimators=2000)

    model.fit(
        tr_x, tr_y,
        eval_set=[(va_x, va_y)],
        callbacks=[lgb.early_stopping(100)]
    )

    oof[va] = model.predict(va_x)
    preds += model.predict(test_x) / 5


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014714 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1368
[LightGBM] [Info] Number of data points in the train set: 504000, number of used features: 14
[LightGBM] [Info] Start training from score 62.482335
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1611]	valid_0's rmse: 8.75873
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013132 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1368
[LightGBM] [Info] Number of data points in the train set: 504000, number of used features: 14
[LightGBM] [Info] Start training from score 62.502155
Training until validation scores don't improve for 100 r

In [7]:
sub["exam_score"] = preds

sub.to_csv("submission.csv", index=False)

print("Submission file created!")
print(sub.head())


Submission file created!
       id  exam_score
0  630000   71.193179
1  630001   71.227196
2  630002   88.199096
3  630003   56.113277
4  630004   48.101341
